In [1]:
import findspark 

In [2]:
findspark.init("/home/efe/spark-3.5.0-bin-hadoop3")

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc


In [4]:
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/28 20:48:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
ssc = StreamingContext(sc,10)
sqlContext = SQLContext(sc)# for query later

/home/efe/spark-3.5.0-bin-hadoop3/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
/home/efe/spark-3.5.0-bin-hadoop3/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

In [7]:
lines = socket_stream.window(20)

In [8]:
from collections import namedtuple# tuple where each element actually is assigned a name, so it has fields
fields = ("tag","count")#main idea is count the actual hashtags in the tweets
Tweet = namedtuple("Tweet",fields)# so every window we get, we're going to take a hashtag of the tweets themselves

In [9]:
lines.flatMap(lambda text: text.split(" ") ) \
.filter(lambda word: word.lower().startswith("#") )\
.map(lambda word: (word.lower(),1) )\
.reduceByKey(lambda a,b: a+b)\
.map( lambda rec: Tweet(rec[0],rec[1]) )\
.foreachRDD(lambda rdd: rdd.toDF().sort(desc("count"))\
.limit(10).registerTempTable("tweets"))

In [10]:
ssc.start()